In [3]:
import os
import numpy as np

import pandas as pd

import tensorflow as tf

import random



In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
class CONFIG:
    SEED = 42
    CLASSIFIER_LR = 3e-4
    EPOCH = 200
    BATCH_SIZE = 8

    SR = 16000
    DURATION = 20 # ms
    FRAME_SIZE = 320

    N_FFT = 64 # (SR*4ms/1000)
    HOP_LENGTH = 32 # (SR*2ms/1000)
    VOICE_DIR = './cv-corpus-19.0-2024-09-13/ko/clips/'
    NOISE_DIR = './ESC-50-master/audio/'

In [6]:
def periodic_hann_window(window_length, dtype):
    return 0.5 - 0.5 * tf.math.cos(2.0 *
                                   np.pi *
                                   tf.range(tf.cast(window_length, tf.float32)) /
                                   tf.cast(window_length, tf.float32))

In [7]:
def wave2log_mel_spectrogram(wave):
    signal_stft = tf.signal.stft(tf.cast(wave, tf.float32),
                                 frame_length=640,
                                 frame_step=320,
                                 fft_length=1024,
                                 window_fn=periodic_hann_window)
    # print(signal_stft.shape) # (49, 513)

    spectogram = tf.abs(signal_stft)

    linear_to_mel = tf.signal.linear_to_mel_weight_matrix(80,
                                signal_stft.shape[-1],
                                16000,
                                300.0,
                                4000.0)
    mel_spectrogram = tf.tensordot(spectogram, linear_to_mel, 1)
    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-12)
    return log_mel_spectrogram

In [8]:
def representative_data_gen():
    valid_df = pd.read_csv('valid_dataset.csv')
    for _, row in valid_df.iterrows():
        path = row['path']
        label = row['label']

        audio_raw = tf.io.read_file(path)
        wave, sr = tf.audio.decode_wav(audio_raw, desired_channels=1)
        wave = tf.squeeze(wave, axis=-1)
        # print(wave.shape) (16000, )
        log_mel_spectrogram = wave2log_mel_spectrogram(wave)
        log_mel_spectrogram = np.expand_dims(log_mel_spectrogram, axis=-1)
        log_mel_spectrogram = np.expand_dims(log_mel_spectrogram, axis=0)

        # print(log_mel_spectrogram.shape) 

        yield [log_mel_spectrogram.astype(np.float32)]


In [9]:
tf.debugging.set_log_device_placement(True)

In [22]:

model = tf.keras.models.load_model('./vad_lstm_v3_29.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter = tf.lite.TFLiteConverter.from_saved_model('./denoiser')

converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32


converter.representative_dataset = representative_data_gen
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8,
# tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
#converter._experimental_lower_tensor_list_ops = False

converter.experimental_new_converter = True
#converter.allow_custom_ops = True
#converter.experimental_enable_resource_variables = True

# 모델 변환
tflite_model = converter.convert()

# TF Select ops: TensorListReserve, TensorListSetItem, TensorListStack 지원하지 않는다
# https://www.tensorflow.org/lite/guide/op_select_allowlist?hl=ko&_gl=1*2zbz7k*_up*MQ..*_ga*MTU4NzE4MDgzMS4xNzMyMjY2NTQx*_ga_W0YLR4190T*MTczMjI2NjU0MS4xLjAuMTczMjI2NjU0MS4wLjAuMA..
# raw ops로는 지원하나, tflite로는 지원하고 있지 않음

# https://www.tensorflow.org/lite/guide/ops_compatibility?hl=ko&_gl=1*1747d60*_up*MQ..*_ga*MTU4NzE4MDgzMS4xNzMyMjY2NTQx*_ga_W0YLR4190T*MTczMjI2NjU0MS4xLjAuMTczMjI2NjY2Mi4wLjAuMA..
# # CUSTOM: 다음 TensorFlow Lite 연산도 제공되지만 사용자 정의 모델에 사용할 준비가 되지 않았습니다.
with open('vad_lstm_29.tflite', 'wb') as f:
    f.write(tflite_model)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:

Executing op StaticRegexFullMatch in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Select in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StringJoin in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShardedFilename in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarIsInitial

INFO:tensorflow:Assets written to: /tmp/tmp_v_1s7b8/assets


Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localh

2024-11-26 10:40:32.294978: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-11-26 10:40:32.295000: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-11-26 10:40:32.295107: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp_v_1s7b8
2024-11-26 10:40:32.302898: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2024-11-26 10:40:32.302913: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmp_v_1s7b8
2024-11-26 10:40:32.329476: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2024-11-26 10:40:32.389807: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmp_v_1s7b8
2024-11-26 10:40:32.435332: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 140224 

ConverterError: /media/insung/새 볼륨/gamba/vad/tf29/lib/python3.10/site-packages/tensorflow/python/saved_model/save.py:1325:0: error: 'tf.TensorListReserve' op requires element_shape to be static during TF Lite transformation pass
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
/media/insung/새 볼륨/gamba/vad/tf29/lib/python3.10/site-packages/tensorflow/python/saved_model/save.py:1325:0: error: failed to legalize operation 'tf.TensorListReserve' that was explicitly marked illegal
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
<unknown>:0: error: Lowering tensor list ops is failed. Please consider using Select TF ops and disabling `_experimental_lower_tensor_list_ops` flag in the TFLite converter object. For example, converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]\n converter._experimental_lower_tensor_list_ops = False
